In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from nova import Nova
from nova.types import Pose
from nova.actions import ptp, jnt

In [2]:
os.environ["CELL_ID"] = "cell"

## 🤖 Show Robot-Pad

In [ ]:
from IPython.display import IFrame

IFrame(src="/cell/robot-pad", width=700, height=600)

## ⚙️ Configure Robot

In [ ]:
nova = Nova()
cell = nova.cell()
controllers = await cell.controllers()
controller = controllers[0]
motion_group = controller[0]
tcp = (await motion_group.tcp_names())[0]

## 🔩 Get Robot Props & State

In [ ]:
state = await motion_group.get_state(tcp)
state

In [ ]:
tcp_pose = await motion_group.tcp_pose(tcp)
tcp_pose

## 📝 Perform Pose Transforms

In [ ]:
current_pose = await controller[0].tcp_pose(tcp)
# Get current TCP pose and offset it slightly along the x-axis
target_pose = current_pose @ Pose((10, 0, 0, 0, 0, 0)) # or current_pose.transform(Pose((10, 0, 0, 0, 0, 0))
print(f"Current\n{current_pose}\n\nTarget\n{target_pose}")

## 🎢 Plan a Motion

In [ ]:
async with controller[0] as motion_group:
    home_joints = await motion_group.joints()
    current_pose = await motion_group.tcp_pose(tcp)
    target_pose = current_pose @ Pose((1, 0, 0, 0, 0, 0))

    actions = [
        jnt(home_joints),
        ptp(target_pose),
        jnt(home_joints),
        ptp(target_pose @ [200, 0, 0, 0, 0, 0]),
        jnt(home_joints),
        ptp(target_pose @ (300, 0, 0, 0, 0, 0)),
    ]

    plan_response = await motion_group.plan(actions, tcp)
    print(plan_response)

## 🏃🏽‍♀️ Execute Motion

In [ ]:
async with controller[0] as motion_group:
    home_joints = await motion_group.joints()
    current_pose = await motion_group.tcp_pose(tcp)
    target_pose = current_pose @ Pose((1, 0, 0, 0, 0, 0))

    actions = [
        jnt(home_joints),
        ptp(target_pose),
        jnt(home_joints),
        ptp(target_pose @ [200, 0, 0, 0, 0, 0]),
        jnt(home_joints),
        ptp(target_pose @ (300, 0, 0, 0, 0, 0)),
        jnt(home_joints),
        ptp(target_pose @ Pose((300, 0, 0, 0, 0, 0))),
        jnt(home_joints),
        ptp(target_pose @ Pose((400, 0, 0, 0, 0, 0))),
        jnt(home_joints),
        ptp(target_pose),
        jnt(home_joints),
    ]

    joint_trajectory = await motion_group.plan(actions, tcp)
    await motion_group.execute(joint_trajectory, tcp, actions=actions)